In [1]:
# This is for speech recognition (needs internet)
!pip install SpeechRecognition pydub

# This is for using a cool AI model from HuggingFace (can work offline kinda)
!pip install transformers torchaudio soundfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
# Use this to upload from your computer
from google.colab import files
uploaded = files.upload()  # pick your file (like audio.wav)

# Get the file name
audio_path = list(uploaded.keys())[0]


Saving MyRecording.webm to MyRecording (1).webm


In [7]:
import speech_recognition as sr
from pydub import AudioSegment

# so like, if the file is mp3, we gotta change it to wav coz google doesn’t like mp3s
if audio_path.endswith('.mp3'):
    sound = AudioSegment.from_mp3(audio_path)
    audio_path = 'converted_audio.wav'  # just naming it whatever
    sound.export(audio_path, format='wav')

# ok now if it’s a .webm file (some random format from phone maybe?), we try to convert that too
elif audio_path.endswith('.webm'):
    print(f"trying to change {audio_path} to wav lol...")
    try:
        # this needs ffmpeg i think... colab mostly has it but if not, gotta install it
        # like this maybe -> !apt-get install ffmpeg (do that separately if errors)
        sound = AudioSegment.from_file(audio_path, format="webm")
        audio_path = 'converted_audio.wav'
        sound.export(audio_path, format='wav')
        print("woohoo converted it ")
    except Exception as e:
        print("uh oh couldn’t convert the webm file. error was:")
        print(e)
        # not stopping the code... just moving on and praying it works

# now let’s try to hear what the person said in the audio
r = sr.Recognizer()

with sr.AudioFile(audio_path) as source:
    print("listening to the audio... hang tight")
    audio = r.record(source)  # grabbing the whole thing

# now we’ll use google’s thing to figure out the words
try:
    print("sending it to google... fingers crossed")
    text = r.recognize_google(audio)
    print("google thinks you said:")
    print(text)
except Exception as e:
    print("yikes couldn’t get the words out of the audio. error below ↓")
    print(e)


trying to change MyRecording (1).webm to wav lol...
woohoo converted it 
listening to the audio... hang tight
sending it to google... fingers crossed
google thinks you said:
hello 1 2 3 testing voice recognition system some random sentences to test today is a beautiful day can you hear what I am saying this is just a random sentence for testing hello how are you today I am testing my Speech to Text converter I wonder how good this system is with longer sentences and pause I'm using Google call app to run this speech recognition code sometimes I talk fast sometimes I talk slow thank you


In [8]:
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Load the brainy AI model and its processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Read the audio file
speech, sample_rate = sf.read(audio_path)

# If sample rate is not what the model wants, fix it
if sample_rate != 16000:
    import torchaudio
    print("Fixing audio sample rate...")
    speech = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(torch.tensor(speech).float()).numpy()
    sample_rate = 16000

# Feed the audio to the model
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values
logits = model(input_values).logits

# Get the most likely words
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print("Wav2Vec2 thinks you said:")
print(transcription.lower())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fixing audio sample rate...
Wav2Vec2 thinks you said:
hallo onto three testing voice recognition system some random sentences protest to day is a beautiful day can you hear what i am saying this is just a random sentence for testing hallo how are you tore i am testing my speech protex canvarder i wonder how good this system is with longer sentences and pause i am using willcolab to run this speech recognition cold sometimes i talk fast sometimes ar talk slow thank you
